<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Подготовить-процессы-обработки-данных" data-toc-modified-id="Подготовить-процессы-обработки-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовить процессы обработки данных</a></span></li><li><span><a href="#Создать-пакет-данных-и-итератор" data-toc-modified-id="Создать-пакет-данных-и-итератор-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Создать пакет данных и итератор</a></span></li><li><span><a href="#Определите-модель" data-toc-modified-id="Определите-модель-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Определите модель</a></span></li><li><span><a href="#Инициировать-экземпляр" data-toc-modified-id="Инициировать-экземпляр-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Инициировать экземпляр</a></span></li><li><span><a href="#Определите-функции-для-обучения-модели-и-оценки-результатов." data-toc-modified-id="Определите-функции-для-обучения-модели-и-оценки-результатов.-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Определите функции для обучения модели и оценки результатов.</a></span></li><li><span><a href="#Разделите-набор-данных-и-запустите-модель" data-toc-modified-id="Разделите-набор-данных-и-запустите-модель-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Разделите набор данных и запустите модель</a></span></li><li><span><a href="#Оцените-модель-с-помощью-тестового-набора-данных" data-toc-modified-id="Оцените-модель-с-помощью-тестового-набора-данных-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Оцените модель с помощью тестового набора данных</a></span></li><li><span><a href="#Тест-на-случайной-новости" data-toc-modified-id="Тест-на-случайной-новости-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Тест на случайной новости</a></span></li></ul></div>

In [1]:
%matplotlib inline


Классификация текста с помощью библиотеки torchtext
==================================

В этом руководстве мы покажем, как использовать библиотеку torchtext для создания набора данных для анализа классификации текста. Пользователи будут иметь возможность


    - Доступ к необработанным данным в качестве итератора
    - Создание конвейера обработки данных для преобразования необработанных текстовых строк в torch.Tensor, который можно использовать для обучения модели.
    - Перемешивайте и повторяйте данные с помощью `torch.utils.data.DataLoader
    
<https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader>`__


Загрузка данных
-----------------------------------

In [2]:
import torch
import pandas as pd

In [3]:
# Список категорий в тестовых данных
# pd.read_pickle('../data/input/test_data_4300_rows.pkl')['category_id'].drop_duplicates().values

In [4]:
df = pd.read_pickle('../data/input/test_data_all_rows.pkl')[['category_id', 'item_name']]
df[df.item_name == 'Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 900мл']

,category_id,item_name
43146,78,Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 9...


In [5]:
def open_data(path, cat_dict = {}):
    df = pd.read_pickle(path)[[
                            'category_id', 'item_name']]
#     df = df[~df.category_id.isin([0, 133, 3, 177, 43, 18, 120, 45, 2, 49, 66, 30, 118, 7])]
    if cat_dict == {}:
        cat_num = 0
        for category_id in  df.category_id.drop_duplicates():
            cat_dict[category_id] = cat_num
            df.loc[df.category_id == category_id, 'category_id_new'] = cat_num
            cat_num += 1
        return df, cat_dict
    else:
        df['category_id_new'] = df.category_id.map(cat_dict)
        return df
    

In [6]:
train_df, cat_dict = open_data('../data/input/test_data_all_rows.pkl')
train_data = train_df[['category_id_new', 'item_name']].to_numpy()
print(cat_dict)
print(train_data.shape)
train_data

{78: 0, 71: 1, 70: 2, 84: 3, 69: 4, 68: 5, 40: 6, 203: 7, 79: 8, 7: 9, 117: 10, 80: 11, 83: 12, 53: 13, 73: 14, 81: 15, 0: 16, 12: 17, 85: 18, 77: 19, 145: 20, 2: 21, 204: 22, 139: 23, 38: 24, 74: 25, 130: 26, 75: 27, 49: 28, 45: 29, 133: 30, 76: 31, 82: 32, 19: 33, 43: 34, 51: 35, 61: 36, 177: 37, 118: 38, 92: 39, 36: 40, 30: 41, 167: 42, 66: 43, 52: 44, 107: 45, 37: 46, 3: 47, 72: 48, 62: 49, 50: 50, 120: 51, 42: 52, 150: 53, 57: 54, 6: 55, 140: 56, 101: 57, 163: 58, 20: 59, 103: 60, 4: 61, 31: 62, 67: 63, 27: 64, 29: 65, 114: 66, 102: 67, 115: 68, 26: 69, 35: 70, 39: 71, 13: 72, 9: 73, 128: 74, 60: 75, 41: 76, 138: 77, 11: 78, 100: 79, 24: 80, 96: 81, 109: 82, 106: 83, 56: 84, 143: 85, 105: 86, 90: 87, 1: 88, 108: 89, 164: 90, 55: 91, 111: 92, 58: 93, 54: 94, 97: 95, 46: 96, 121: 97}
(48267, 2)


array([[0.0, 'Молоко 3,2%,шт'],
       [1.0, 'Компот из изюма, 114 ккал'],
       [1.0, 'Макаронные изделия отварные (масло сливочное), 240 ккал'],
       ...,
       [16.0, 'Пиво светлое "Халзан" 4,5 % об, пл/б. 1,5 л(шт)'],
       [52.0, 'Экспресс педикюр'],
       [3.0, 'Конфеты Харитоша 1кг мол. ваф Яшкино']], dtype=object)

In [7]:
test_df = open_data('../data/input/test_data_6350_rows.pkl',cat_dict = cat_dict)
test_data = test_df[['category_id_new', 'item_name']].to_numpy()
test_data

array([[0, 'Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 900мл'],
       [11, 'Мандарины с листиком 1кг'],
       [3, 'Хлеб/бул изделие 200г отруби  ХБЗ №2'],
       ...,
       [7, 'Пакет'],
       [19, '230Г СОУС КАЛЬВЕ С ЛЕСНЫМИ ГРИ'],
       [7, 'ЗЕЛПМ-КА32Х62Х17']], dtype=object)

In [8]:
# Test load Data
item_name_train = test_df.loc[test_df.item_name == 'Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 900мл']
item_name_test = train_df.loc[
            train_df.item_name == 'Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 900мл']
print(item_name_train, '\n', item_name_test)



          category_id                                          item_name  \
27702571           78  Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 9...   

          category_id_new  
27702571                0   
        category_id                                          item_name  \
43146           78  Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер 2.5% п/пак 9...   

       category_id_new  
43146              0.0  


Подготовить процессы обработки данных
---------------------------------

Мы пересмотрели самые основные компоненты библиотеки torchtext, включая словарь, векторы слов, токенизатор. Это основные строительные блоки обработки данных для необработанной текстовой строки.

Вот пример типичной обработки данных НЛП с помощью токенизатора и словаря. Первым шагом является создание словаря с помощью необработанного набора обучающих данных. Пользователи могут иметь собственный словарь, задав аргументы в конструкторе класса Vocab. Например, минимальная частота ``min_freq`` для включения токенов.

In [9]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

tokenizer = get_tokenizer('basic_english')
counter = Counter()
for label, line in train_data:
    counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=1)

Блок словаря преобразует список токенов в целые числа.

::

    [vocab[token] for token in ['here', 'is', 'an', 'example']]
    >>> [476, 22, 31, 5298]

Подготовьте конвейер обработки текста с токенизатором и словарем. Конвейеры текста и меток будут использоваться для обработки строк необработанных данных из итераторов набора данных.



In [21]:
def text_pipeline(text):
    return [vocab[token] for token in tokenizer(text)]    

In [22]:
# text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x)

Текстовый конвейер преобразует текстовую строку в список целых чисел на основе таблицы поиска, определенной в словаре. Конвейер меток преобразует метку в целые числа. Например,

::

    text_pipeline('here is the an example')
    >>> [475, 21, 2, 30, 5286]
    label_pipeline('10')
    >>> 9




Создать пакет данных и итератор
--------------------------------

`torch.utils.data.DataLoader <https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader>`__
рекомендуется для пользователей PyTorch (учебник `here <https://pytorch.org/tutorials/beginner/data_loading_tutorial.html>`__).

Он работает с набором данных в стиле карты, который реализует протоколы ``getitem()`` и ``len()`` и представляет карту от индексов/ключей до образцов данных. Он также работает с повторяющимися наборами данных с аргументом случайного выбора ``False``.


Перед отправкой в модель функция ``collate_fn`` работает с пакетом образцов, сгенерированных из ``DataLoader``. Входом для ``collate_fn`` является пакет данных с размером пакета в ``DataLoader``, и ``collate_fn`` обрабатывает их в соответствии с конвейерами обработки данных, объявленными ранее. Обратите внимание на это и убедитесь, что ``collate_fn`` объявлен как def верхнего уровня. Это гарантирует, что функция доступна каждому исполнителю.

В этом примере текстовые записи в исходном вводе пакета данных упаковываются в список и объединяются в один тензор для ввода ``nn.EmbeddingBag``. Смещение - это тензор разделителей, представляющий начальный индекс отдельной последовательности в текстовом тензоре. Метка - это тензор, сохраняющий метки отдельных текстовых записей.



In [27]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        print(_text)
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)    

dataloader = DataLoader(train_data, batch_size=8, shuffle=False, collate_fn=collate_batch)

Определите модель
----------------

Модель состоит из `nn.EmbeddingBag <https://pytorch.org/docs/stable/nn.html?highlight=embeddingbag#torch.nn.EmbeddingBag>`__ слой плюс линейный слой для целей классификации. ``nn.EmbeddingBag`` с режимом по умолчанию «mean» вычисляет среднее значение «мешка» вложений. Хотя текстовые записи здесь имеют разную длину, модуль nn.EmbeddingBag здесь не требует заполнения, поскольку длина текста сохраняется в смещениях.

Кроме того, поскольку ``nn.EmbeddingBag`` накапливает среднее значение по
вложения на лету, ``nn.EmbeddingBag`` может улучшить
производительность и эффективность памяти для обработки последовательности тензоров.

![](../_static/img/text_sentiment_ngrams_model.png)





In [12]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

Инициировать экземпляр
--------------------

Набор данных ``AG_NEWS`` имеет четыре метки, поэтому количество классов равно четырем.

::

   1 : World мир
   2 : Sports спорт
   3 : Business Бизнес
   4 : Sci/Tec Наука/Техника

Мы строим модель с размером встраивания 64. Размер словаря равен длине экземпляра словаря. Количество классов равно количеству этикеток,




In [13]:
num_class = len(cat_dict)
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [14]:
num_class

98

Определите функции для обучения модели и оценки результатов.
---------------------------------------------------------




In [15]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()
    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
#         print(sorted(label))
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

Разделите набор данных и запустите модель
-----------------------------------

Поскольку исходный ``AG_NEWS`` не имеет действительного набора данных, мы разделяем обучение
набор данных в обучающие/действительные наборы с коэффициентом разделения 0,95 (train) и
0,05 (valid). Здесь мы используем
`torch.utils.data.dataset.random_split <https://pytorch.org/docs/stable/data.html?highlight=random_split#torch.utils.data.random_split>`__
в базовой библиотеке PyTorch.

`CrossEntropyLoss <https://pytorch.org/docs/stable/nn.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss>`__
критерий объединяет в одном классе ``nn.LogSoftmax()`` и ``nn.NLLLoss()``.
Это полезно при обучении задаче классификации с помощью классов C.
`SGD <https://pytorch.org/docs/stable/_modules/torch/optim/sgd.html>`__
в качестве оптимизатора реализует метод стохастического градиентного спуска. Начальный
скорость обучения установлена на 5.0.
`StepLR <https://pytorch.org/docs/master/_modules/torch/optim/lr_scheduler.html#StepLR>`__
здесь используется для регулировки скорости обучения по эпохам.




In [28]:
from torch.utils.data.dataset import random_split
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # скорость обучения
BATCH_SIZE = 16 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
# train_iter, test_iter = AG_NEWS()
train_dataset = list(train_data)
test_dataset = list(test_data)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

# print(train_dataset)
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

Сок ФН 200мл в ассорт
 F.H.Полосатик желт.суш. 70г
Печенье мягкое Шоколадное
ТОМАТЫ РОЗОВЫЕ РОСС
КД/МИНДАЛЬ ОБЖ 50ГР
Толстовка (J2F033,M,5900)
Хлопья для завтрака кукурузные 750г.
 Десерт"КидБокс"20гр касса
7 Акция Чизкейк манго - 129 руб.
мороженое 2 вкуса
Сыр Коса копченая БЗМЖ 1 кг ООО Молзавод Новый Республика Адыгея
Камбала вяленая в вак/уп. 70 гр
 Крупа манная"Оптсервис"800гр
Носки мужские DMDBS "А18-061" махровые
1 775 878 Носки всесезонная CALVIN KLEIN
Мини - чебуреки  120/40....
СЫРНИКИ 100 Т
Напиток МОЯ СЕМЬЯ ананас манго маракуйя 0.95л
Лайм весовой Коста-Рика
ГРЕЧЕСКИЙ ЧЕРНИКА
Соусы "Хайнц" в ассортименте
Освежитель Glade Эф Свежести Родниковая свеж 275мл
СУП ПЮРЕ ИЗ ОВОЩЕЙ
4 Сыра 30 см
Ацикловир мазь д/наруж примен 5% 10г
 МОЛОЧ.ПОЛЯНА Кефир 1% ПЭТ 450г
Напиток Фруктовый Фейерверк 0,75л барбарис газ с/б
Набор гелевых ручек 24шт имп
Станок д/брит Gillette Venus Snap Emb+1кас
ВЕТЧ ВЕТЧИНА ПО-ЧЕРКИЗОВСКИ ЦО В
Груши весовые Конференц
Сочень с творогом 90 гр
КЛЮЧ ТОРЦЕВОЙ Т-ОБРА

ПРОДУКТ ЙОГУРТНЫЙ НЕЖНЫЙ ЛЕГКИЙ С СОКОМ КЛУБНИКИ 0,1% КАМПИНА 95Г
1/1 Оксолиновая мазь 0,25% 10г
Мини чиабатта кукурузная Panus
5  08.19 Марципа/GROND/ из а
Сметана ДАЛЬНЯЯ ФЕРМА 20% 200г
Головы Куриные н/калибр  охл подл Весовая
3  08.19 Хлебцы /Block/с пом
ПЦ/ЛОЖКА ОДНОР СТОЛ,
Колпак Торт яркий 6шт
Минтай жареный в яйце
нестле каша безмолочная 200г овсяная
Пицца С ветчиной
Рогалик с персиком 2 шт.
Батон, хлеб
Шапка (G215,54-58,9100)
Лимонад Майкопский Груша 0,5 л с/б
Приправа Жар Востока для маринования овощей м/у 15 г
Зефир СЛИВОЧНАЯ НЕЖНОСТЬ
0415 0415 Наложение шва посттравматическое, косметический шов (е
Оплата электроэнергии
Кабачки Турция
Вареники с картоф. и кваш. капустой,шт
Джемпер BSW трикотаж белый/разноцветный 10-12л/152
СМС АРИЭЛЬ АВТОМАТ ГЕЛЬ-КАПСУЛЫ КОЛОР 13Х27Г
ПАРАЦЕТАМОЛ 500мг N20 таб. Обновление Реневал ПФК
Суп Уха по-кахетниски б/л
13Г Ж/Р WRIGLEY'S JUICY FRIUT
Напиток газированный Кока-Кола
3/30-НИМЕСИЛ (пак.) 0.1 г/2г N30
Шоколад Озера С Грушей 50Г
Масло персиков

Набор "Супер сет"
 Пирог блинный с творогом
воздушный зефир для какао"ТРИ КОТА" 15г
Салат сельдь под шубой_ от бре
 Тройник ПП 20 мм&
3  02.20 Изюм су/SEEBE/Изюм
14.1 Жареная лапша с овощами
Мешки для мусора 35 л "Стандарт", толщина 6 мкм, рулон 30 шт 
КАРТОФЕЛЬ (СЕТКА) 10 КГ
Хлебные палочки OSTERIA ITALIA
СОСИСКА В ТЕСТЕ 120 ГР
Семечки черные Джинн 70г
РЕШЕТКА ДЛЯ ГРИЛЯ И БАРБЕКЮ ДЛЯ СТЕЙКА ТРЕХСЕКЦИОННАЯ АРТ. 3-24В
Пиво светлое (пастеризованное) "Большая Кружка Чешское" ПЭТ, Ростов 4% 1.3л. ООО
Сгон чёрный СГЧ15 ДУ15<>
Скотч двусторонний 19мм*10м полипропилен Klebebander (24шт. уп/96шт) шт
Салат из морской капусты с луком, 200 г,шт
Солянка мясная*
W847-S Носки для спорта (1 пара) Wilson Sport socks (1 pairs) р.M
Морож Бон Пари Джангли ван/клубн 45г Нестле
ТОМАТЫ СЛИВОВИДНЫЕ ЖЕЛТЫЕ 1 КГ КРАСНОДАРСКИЙ КРА
Кефир КрымсМолочник 1% _ 1л ГОСТ-2012
Пепси-Кола 1 л бут
Вупи пай Шоколад
Мыло Ватика Черный тмин 115г (2021.10)
Гель д/стирки универсальный 1,2л Асс
ИР Винилин 25г
0,5Л ПИВО ЧЕРНОВАР

Сок разливной 0,2 л
Винегрет с квашеной капустой 1
Осетр крупный Premium горячего
Кофе Энвол раствор сублимир 50г м/у
Голубцы ленивые с говядиной 200/50 гр
4) Каша гречневая
Хинкали с говядиной 3 порц.
б1 Кофе Капучино 1 шт
Грунт "Ceresit" универсальный 5л., СТ17
Жил. ком. услуги
САЛФ/ВЛАЖ/АНТИБ
Кус-кус  150  гр
Мороженое 48 копеек Эскимо с малиной 90мл
Лента декоративная самоклеящаяся тканевая , полиэстер, 1,8см/2м, 6 цветов
Крендельки "Бре
Вафли "Дарлетто" клюква 123гр
Ф/К СЕЛ В СЫР СОУСКД
 Блин"С пыл с жар"ветч/сыр360г
Крючки Osprey OS- Лещ и Голавль (№11)
Майка FA(100шт)
Вилка б/з бел. Universal A113  ^++
МИКРОЛАКС 5мл N4 клизма МакНил Мэньюфэкч
Молоко стерилизованное Первозд
Молоко Российское 2.5% 0,5л KОРОБKА х15
Облепиха Просто Азбука 350 г Р
1/1 Валокордин капли 20мл
A18ADEPAB11-Z4 128 Брюки для мальчиков Boy's Pants темно-синий р.128
ПОМИДОРЫ  МАТВЕЕВСКИЕ СЛИВКИ  1 КГ  КОД 378
Клей 30мл блистер Момент 88
Бутерброд Биг №14 2 сосиски,сыр,кетч.,майон.,чесн. 240г
ТЕТРАДЬ 12-24Л 12

 Хрустящие +Колечки-кар.нач
Салат Сельдь под шубой вес. Собственное пр-во
Водка Мороша На минеральной воде Карелии Номер 2 40% 0,5
Блинчики с ветчиной и сыром
Начинка яблочная 1 порц.
/ЗЕЛЕНЬ 50Г
Альпен Голд Арахис/карамель/дражже молочный шоколад 160гр
Бутоньерка ягодки простые (красный) шт
стм рыбный жир капс. 0.33мг n100 с э
Вода питьевая Раифский Источник 0,5л газ ПЭТ
Отвод канализационный  50х45  (100 шт.уп.)
Егермайстер 50мл.
Ланч №2
2) Оладьи дрожжевые со сгущеным молоком    100/20 гр.
 Печ.пес"Твикс"55г
Петрушка 20г
Антиперспирант-карандаш Сухость пудры RU
ЯЙЦО КУРИНОЕ С1 ВАРА
Картофель запеченный с розмари
Слива красная,кг
ЧИТОС СМЕТАНА ЛУК
Рамэн с цыпленком, маринованным бамбуком и грибами шиитаке
МОРОЖЕНОЕ РОЖОК С ГОЛУБИКОЙ ЛОТТЕ 140Г
Шаньга с картошкой
ECOBOTANIKA ЗЕЛ ЧАЙ,
Круассан с малиной 90г
Колбаски Охотничьи 1кг
NESCAFE 3В1 16Г
1/1 Салициловая мазь 2% 25г №1
Кофе Капучино в ассортим Якобс 220
Кофе американо 400мл
Горчица ПОВАРЕНОК д/пак 150г
Горячая сковорода с курице

...

Датчик температуры WS2697
Мороженое Бодрая корова Эскимо 70г
ПЛЮШКА НОВОМОСКОВСКА Я С САХАРОМ 150 Г ИП СТРЕКА
Дуовит, држ №40
АБРИКОСЫ 1кг
Мороженое Вита Магия плом джем перс/манго в/ст 95г
Носки муж р27 черный ЭТО
Паста Тальятелле в сливочном соусе
Салат Рыжик (морковь,сыр,яйцо,лук,майонез), 225 ккал
ЙОГУРТ КАМПИНА НЕЖНЫЙ 320Г ПЕРСИК 1,2%
Полосат БИРКА желт40 
Вода Питьевая "Капля Моря" н/газ 1,5л
Пиво АМСТЕЛ ПРЕМИУМ ПИЛСЕНЕР све
Свинина запеченая с овощами,180гр
Сок "Я" в ассорт. 0,2
Фарш гов. Традиционный охл 400г ГЗМС Фермерский бычок Мираторг Россия
Контейнер д/анализов, 100 мл ЕЛМД
ОСВ БАЗ АЭР МОРСКОЙ
Капучино 240мл
АЦИДОФИЛИН 3.2% 200Г
Шарф (,no size,1100)
Чабрец 1,5г
12  3ПАРЫ НОСКИ Ж/LOVES/LSHLS
СОЧНИ 1 ШТ
GOURMET для кошек консервы 85г Gold Форель с овощами кусочки в подливке
ПЮРЕ/Я/САМ/БАНАН
Молоко 3,2% Кудымкар
Натрия хлорид р-р д/инф 0,9% 200мл
Fresh апельсин 250 мл
Плинтус потолочный для панелей 3 м. шт
              ПЕРЧАТКИ ХОЗЯЙСТВЕНН
682. Стакан для горячих напитков 2

Драже M&M`s Макси с арахисом 7
Пир. с Картофелем
КАПУСТА БЕЛОКОЧАННАЯ 1 КГ РОСТОВСКАЯ ОБЛ.
Кофе Латте Малый Эден 18
Бублик сдобный 460г /Республика Хакасия
Хлеб ремесленный ржаной 100% на закваске/Европейский
зубная паста Сплат Проф 100мл в ассортименте шт
МЫЛО ТУАЛЕТНОЕ СЭЙФГАРД АКТИВ КЛАССИЧЕСКОЕ 5*75Г
диоксидин р-р д/и 1% 10мл амп n10
Кофе Капучино 0.2л, шт
Стейк Денвер гов. охл 290г SKIN Black An
Хлеб Батон  400 гр
дыши пластырь для ингаляций n5
Сухарики Бородинские черные
Компот 15 р.
КЛЕЙ ПВА-М FANCY,85Г
Пивной нап Миллер Дженьюин Драфт 4,7% 0,33л ст/б
Овощная горка 100 гр.
Пастила
Кейк попс в асс.
Филе куры в сливках
молоко Арта 3,2% Полдень 1л
тонк кар д/бро ESPRESO MBP07
НАПИТОК СВЯТОЙ ИСТОЧ
ДЖИНСЫ (2834)
Кефир детский 3,2%, 210 г,шт
Напиток Пепси Кола Лайм 1.5л пэт
Ролл Канада
1) Чешский Кабанчик Светлое пшеничное н/ф 4,4% Комплект 1,5
Буженина по-баварски от шефа
СЕМЕЧКИ  150Г
Джемпер (,XXL,50M0)
АТЕНОЛОЛ 100МГ. №30 ТАБ. /RENEWAL/
Раф Ежевичный Пай 300 мл
Пиво Варим сусло 1,

СИСТЕМА ДЛЯ ПЕРЕЛИВАНИЯ РАСТВОРОВ
КРЕМ НИВЕЯ МЕН 75МЛ Ж/Б
Форма д/выпечки прямоугол., 32*16 см ИМП
Капучино-150
Белье (U1G055,L,7000)
Блин с говядиной в луковом соусе 1 порц.
КУКУРУЗНЫЕ ШАРИКИ
Молоко 2,5% мкс 1000г
Витаминный салат
Горох колотый Россия
Филе грудки индейки охлажденно
Пир Песочное гл помадк 80гр
Плов из мякоти кур (ШХ)
Ж/р орбит
КУКУРУЗА 340Г HEINZ
Крылья Акашево охл. подложка вес. 1 кг
Скумбрия х/к (ИП Ковалев) (4969)
Начинка д\б зелень 5г шт)
Краюшки Ржано-Пшеничные 100 гр
Хлеб Пряный 0,4г нарезка
Сигареты Паркер и Симпсон Kрасный 100 МРЦ 90
с-т сельд под шубой, шт.
Футболка (W302,L,90M0)
Носки женские (3 пары)
1/1 Глицелакс супп. рект. д/дет
Молоко ДЕРЕВЕНСКОЕ МОЛОЧКО пастер отборное 3.5-5.4% пэт 930мл
Шоколадный батончик BOUNTY 55г KT
БЗМЖМороженое Чистая лин 100г<>
Kомпот
Чай Фьюз 0,5л в ассорт.
Мини Тортики 1кг
911 пчелиный яд гель-бальзам для тела 100мл
БУЛГУР С ТЫКВОЙ И ШПИНАТОМ 400Г
(БиМ)(БР) Стартовый (300)(шт, )
Занзибар
2  Футболка для /RHS/30-1.
C: ЖЕВАТЕЛЬН

ЯЙЦА КУР С1,КД 30ШТ
КП/ЯЙЦО ДЕРЕВЕНСКОЕ
Сок в асс 0,2л
6) Филе Прованс
Ветчина из бедра индейки п/а 400г Даурия 1/30
Орех свиной Традиционный Скиф
Шаньга с картофелем 100г, 218 ккал (завод)
Салат из  б/капусты  с морковью 100
Водка "Финский Стандарт Оригинальная" 0.5л 38%
D06-02 36  Чешки для фитнеса и танцев Fitness and dance gymnastic shoes белый р.36
 Ролик д/всех видов ткани 20лист СМ
Лампа светодиодная ECO T75 таблетка 8Вт 720Лм 230В 4000К GX53 IE
Арахисовая паста классическая,шт
Вода питьевая Аква минерале газ 2л пэт
ТРЯПКА Д/ПОЛА МИКРОФ ARO
АИ-92-К5 АИ-92-К5 (3 ТРК)
КОКА-КОЛА 2,0Л
Сыр Российский Александровский БЗМЖ вес.
тарелка суповая  Карин Лайт Тю
Жев. рез. Орбит белосн. нежн. мята 13,6г
Манго жёлтое Крупное
Печень минтая натуральная (0,190)
Кешьюб/обжарДарыприр15
 Кокт/МорепМирамар/рассол*300гр
Пирожок с рисом и яйцом
"Жар -семечка"соленые 70г
Укроп/петрушка 1шт
12 Пиво   "Алафузов пшеничное"
8  ЖАКЕТ ЖЕН/PUNT /
 Хачапури слоеные Атолл 4шт*50г фасовка
СЫР ШВЕЙЦАРСКИЙ 50%
пи

KeyboardInterrupt: 

Запуск модели на GPU со следующей распечаткой:

::

    -----------------------------------------------------------
    | end of epoch   1 | time:  0.03s | valid accuracy    0.450 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   2 | time:  0.03s | valid accuracy    0.450 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   3 | time:  0.03s | valid accuracy    0.450 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   4 | time:  0.03s | valid accuracy    0.650 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   5 | time:  0.02s | valid accuracy    0.700 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   6 | time:  0.02s | valid accuracy    0.700 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   7 | time:  0.02s | valid accuracy    0.700 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   8 | time:  0.02s | valid accuracy    0.700 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch   9 | time:  0.02s | valid accuracy    0.700 
    -----------------------------------------------------------
    -----------------------------------------------------------
    | end of epoch  10 | time:  0.02s | valid accuracy    0.750 
    -----------------------------------------------------------



Оцените модель с помощью тестового набора данных
------------------------------------




Проверка результатов тестового набора данных…



In [17]:
print('Проверка результатов тестового набора данных.')
accu_test = evaluate(test_dataloader)
print('точность теста {:8.3f}'.format(accu_test))

Проверка результатов тестового набора данных.
точность теста    0.780


::

    Проверка результатов тестового набора данных.
    точность теста    0.950




Тест на случайной новости
---------------------

Используйте лучшую на данный момент модель и проверьте новости гольфа.




In [24]:
cat_dict_inverted = dict(map(reversed, cat_dict.items()))

def predict(text, text_pipeline = text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        print(text)
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()

ex_text_str = "Сыр ПОШЕХОНСКИЙ 45% вес"

model = model.to("cpu")
predict_cat = predict(ex_text_str, text_pipeline)
print("Это %s категория" %cat_dict_inverted[predict_cat])
exsempl_df = pd.DataFrame(train_data)
print('Примеры:\n', exsempl_df[exsempl_df[0] == predict_cat][1][:50].values)

tensor([  38, 3649,  277,   24])
Это 78 категория
Примеры:
 ['Молоко 3,2%,шт' 'Молоко пастерилиз. т/рекс 2,5 %  1 л. 1/8 БМК'
 'Сметана  Кубанский  Молочник  20%  180гр шт' 'СМЕТАНА 20% 300Г'
 'Молоко Мудрая хозяйка 2,5% пленка 0,8 л БЗМЖ'
 '45Г ГЛ/СЫРКИ РОСТАГРО 20% ВН БЗМЖ' 'Пакет'
 'Биойогурт "Козимель" натуральный,шт'
 'Продукт кисломолочный Тема Биолакт 3,2% ТБА 208г'
 'Йогурт питьевой Асеньевская фе' 'Сыр Российский 1кг 50% Россия БЗМЖ'
 'Сметана 10г, 16 ккал' ' К.Ц.Яйца куриные стол.С2 10шт' 'ЯЙЦО ЭКСТРА С0'
 'ЯЙЦО КУРИНОЕ С1 ВАРА' 'Йогурт фруктовый "Вишня" 2,5%,шт'
 'Молоко Отборн Грин Агро 3,3-4,3% 1л пэт ГМЗ Артем'
 'Молоко ФП пастериз т/топ 2,5 % 1,0л 1/6 Хладокомбинат'
 'МАСЛО 82,5% ЭКОМ.' 'Яйцо кур С2 10шт пл/бл Племптицезавод'
 'Мильфей с клубн.тартаром'
 'БЗМЖ Сыр Сметанковый 45 - 50% 250г кусок Продукты из Елани'
 'Сметана 20%,шт' 'ЯЙЦО СТОЛОВОЕ 1КАТ. ПТИЦЕФАБРИКА ПИОНЕРСКОЕ 10ШТ'
 'Сыр с аром топл мол Черный принц 50% Беларусь'
 'Коктейль детский молочный c шоколадом 2

In [26]:
text_pipeline(['Сыр Мраморный 45% Беларусь', 'Сметана 100г, 159 ккал'])

AttributeError: 'list' object has no attribute 'lower'

In [25]:
# df_test = pd.DataFrame({'item_name' : ['Лук', 'Хрен', 'Суп семи залуп', 'АИ95']})
df_test = pd.read_pickle('../data/input/test_data_6350_rows.pkl')[['category_id', 'item_name']]
df_test['Pred'] = df_test.item_name.apply(predict).map(dict(map(reversed, cat_dict.items())))
df_test[df_test.category_id != df_test.Pred]

tensor([  40, 1375, 1754,  547,   11,    2,   30, 5217, 1011])
tensor([  765,     4, 33169,    17])
tensor([3729,  745,   50, 6154, 2391,  680])
tensor([22097,   100,  2896,  1738,  4231, 11763,  2421,    31,     2,   924])
tensor([  27,  241,  209, 1041])
tensor([1197, 1075])
tensor([ 935, 3223])
tensor([  4, 831,   8,   9,   2])
tensor([  293,   723,    10,   721,  1026, 19518])
tensor([376,  75,   4, 227])
tensor([1032])
tensor([1032])
tensor([ 546, 1388, 3288])
tensor([   21, 38331])
tensor([ 148,  456, 1350,  603,    2])
tensor([1032])
tensor([1032])
tensor([ 1954,   709,   827,     5,    74,   625,     6,    88,     3, 18566,
          244])
tensor([ 882, 1083, 2121])
tensor([5219,    4,  629,  170])
tensor([398, 284,  17])
tensor([  724,  8090,    25,  1133, 15999, 13231])
tensor([ 472, 1961, 1994, 4590, 1012,    2,  327,    2, 1481,   18])
tensor([  79,  208, 9934,  439, 1359])
tensor([13695,    19,     3, 19373])
tensor([ 183, 1386,  477])
tensor([  599,   445, 28724,   431,  

tensor([  33, 1573,    8,    3,   15,   41])
tensor([ 3091,    43,   155,  8517, 23249, 22189])
tensor([752])
tensor([1032])
tensor([30390])
tensor([ 201,    4, 5970,    7,    3,   13,   29])
tensor([1928, 1222, 7552, 5780])
tensor([929, 384,   3,  68])
tensor([41339, 20793,     3,   954])
tensor([  126,   317,   262, 10362])
tensor([1032])
tensor([1734,  464,   37,  536])
tensor([ 139,  909, 2372,  558])
tensor([  348,  1786,     2,  4912,     3, 19047,     3,   755])
tensor([ 136, 2543,    2, 1521])
tensor([1032])
tensor([1032])
tensor([1212,   36, 1930,  902,  347])
tensor([   51,  1460, 27979,   484])
tensor([   23,  1351,   622,  4056, 10894,  5930,    35])
tensor([1032])
tensor([1032])
tensor([ 1595,  1415,     4, 12535,   362])
tensor([602, 416, 129])
tensor([  27, 4610,    4,  980])
tensor([1032])
tensor([1032])
tensor([  75,  358, 1780])
tensor([  21, 1149, 1839])
tensor([1032])
tensor([  828, 31815,   148, 13639])
tensor([31983,   745,     4,  2898,   392,  2320,   341])
tens

tensor([1032])
tensor([ 144, 3865, 7236,    2, 3770])
tensor([1327,  497,   86,    2])
tensor([1032])
tensor([  871, 16071,     5,   633,     6])
tensor([ 1359,   877, 34084,  3671,   682])
tensor([1032])
tensor([  21, 1040])
tensor([  28,  640,   59,   13,    2,  265,    7,    3, 1312,   60])
tensor([ 7645,    90,   373,  1020, 42667,    10, 29998])
tensor([ 566, 5405, 4277, 3291,  539,   61,  264,  779])
tensor([  94,  322, 4772, 6931])
tensor([1032])
tensor([ 160, 2085,   24,    2,   17,   42])
tensor([1605,    4,  361, 1403])
tensor([   51, 33563,   138,   119, 38299])
tensor([1032])
tensor([445, 565])
tensor([3953, 3635])
tensor([   58,  1176,    77, 33227,   412, 25288])
tensor([1032])
tensor([1032])
tensor([  21,  730,   61,  349, 1844, 3727])
tensor([41663,    10, 37620, 33280])
tensor([1962,  434,    4,  588,    3,    9])
tensor([1032])
tensor([   67,   324,     4,   591,   142,  7970, 13931])
tensor([1032])
tensor([17868,  2731,  6873,     4,  2552,  8433])
tensor([22111, 358

tensor([ 136, 1522,    2, 1487])
tensor([3372,    4,  211])
tensor([1374,    3, 1449])
tensor([1032])
tensor([ 869, 1839,   30,  419, 9193])
tensor([   21,  5808, 12445,   240,   273])
tensor([5487, 4607,  524])
tensor([ 559,   23, 4607, 8051,  206,   60,   34])
tensor([   85,     2, 31504, 10369])
tensor([ 296, 1083,  199])
tensor([1032])
tensor([30390])
tensor([1297,   24])
tensor([1032])
tensor([838])
tensor([  443,     2,    31,     2, 35801,   209])
tensor([  832,  7067, 21719,   216,   104,     2,  4173])
tensor([ 179, 7676,  484, 1647,  700])
tensor([1032])
tensor([1453,  447,    2,  394, 2589])
tensor([1032])
tensor([17029,   407,  1410,    48,   581])
tensor([    7,     3,    15,    28, 42244, 34719,  1563])
tensor([ 179, 1256, 9804,    8])
tensor([1032])
tensor([ 531,   66,   29,    2,    5, 1518,    6])
tensor([  35,   72, 1169, 1208, 6904,  985])
tensor([ 901, 1605])
tensor([776])
tensor([1032])
tensor([1831, 1083, 1909])
tensor([2316, 1321, 1162])
tensor([1032])
tensor([  

tensor([ 94, 356,   3, 604, 244])
tensor([ 126,  256, 6008])
tensor([  57,  159,   81,    4, 3201,  117])
tensor([1032])
tensor([1032])
tensor([43285,     8,    29,     2])
tensor([ 1379,   598, 15116,  5551,     2,  1304])
tensor([1032])
tensor([1032])
tensor([ 201, 2768, 1417, 1707,   60])
tensor([1032])
tensor([ 111, 4863, 3263, 1908])
tensor([1689,   16, 2771, 1295,  332])
tensor([6112,  347])
tensor([1032])
tensor([ 643, 5088,   17])
tensor([  40, 1989, 7315,   19,    3,   69,    7,    3,  422, 5231])
tensor([842, 376])
tensor([13506,   201,   485])
tensor([  518,     2,   578,    12,  4176, 30750])
tensor([2463])
tensor([15138,  1882,  5506])
tensor([ 315, 4625])
tensor([1032])
tensor([1030,  701,  394,    8,   22,   71])
tensor([2366,  683, 2596,   18, 2620,  157, 1067, 1295])
tensor([1032])
tensor([ 367,  842, 2997,  607,   17,   24])
tensor([ 126,  317, 4138])
tensor([1048, 1171,    7,    3,   13])
tensor([1032])
tensor([   57,   775,  1633,   159, 31186, 13402,    90])
tensor

...

tensor([35])
tensor([1032])
tensor([643,  81])
tensor([  19, 2539,  106, 4159, 1358,  531,   66,   29,    2,    5, 1518,    6])
tensor([  219,    10, 40465,   263,     3,    24,    22])
tensor([   73, 30606,   557, 11537])
tensor([17868,  2731,  6873,     4,  2552,  8433])
tensor([1540, 4023,   51])
tensor([  133,   125, 14473,    30,    83])
tensor([  66, 3546])
tensor([1032])
tensor([ 1895, 38200, 35768,   106, 14379,     4,  4131])
tensor([  52, 5371,   26,  811,  271])
tensor([   51,  1460, 27979,   484])
tensor([5270,    2,  270,   17])
tensor([1032])
tensor([ 338, 1038,   18,   17])
tensor([ 109, 5087,    4,  462,   10,  409,   12,  246,   90,  373])
tensor([ 132, 1960,   10,   92])
tensor([ 2508,  7908, 25253])
tensor([ 643, 5088,   72,   17])
tensor([1030,  701,  394,    8,   22,   71])
tensor([  7,   3,  11, 153,   2, 108,   2,  43, 285])
tensor([1032])
tensor([  75,   81, 9582,    2])
tensor([  151,  2967,     2,     4, 13472,     3,    88])
tensor([1018,  632])
tensor([ 198,

tensor([1293,  382])
tensor([1032])
tensor([   35,   447,  2382,   247,  8019,     8,     3, 20764])
tensor([ 27, 847])
tensor([ 1393,    16,  1456, 13796,   292])
tensor([  33, 1573,    8,    3,   15,   41])
tensor([20017,   247, 14168,    17])
tensor([  40, 4536,  960, 2845,   19,    3, 8226,    3,  541,   34,   41, 2455,
        4289])
tensor([30412,  5108,     4,   867,    37])
tensor([780,  34])
tensor([2323])
tensor([ 3737,  6397,  1428, 29971,  2874,  4361])
tensor([1298,    5, 4200,    6])
tensor([1032])
tensor([1032])
tensor([3156,    3,  810,   18])
tensor([1030,   16,  502])
tensor([ 139, 4078, 2141])
tensor([  122,   435, 10584,  6109,   175,   117])
tensor([151,   4, 675,  14, 467])
tensor([1867, 1734,  464])
tensor([496])
tensor([  833, 40037, 17546])
tensor([7546,  683,  164, 2596,   18, 2620,  157, 1067, 1295])
tensor([1032])
tensor([1926, 1090,    4, 9678,   53])
tensor([13505,   152,     2,  1965,     2,  8134])
tensor([1032])
tensor([1032])
tensor([593])
tensor([1549

tensor([  77, 1437,  768, 1035, 8614])
tensor([ 331,   41,   34,    4, 3958])
tensor([ 21,   4,  80, 147,  18])
tensor([  387,   454,     4,  1118,  1582,     5,   994,     6,     3, 18089,
          244])
tensor([1032])
tensor([  79,  703, 2815,    2, 6215, 7575,  231])
tensor([   40,  1879,   214,    89,     2,    30, 15050])
tensor([ 4291, 13832,  1016,     7,     3,    13,    41,     5,  5817,     6])
tensor([ 64, 435])
tensor([ 726, 1893])
tensor([ 865,  881, 1351,   37,  188, 5808])
tensor([629, 327,   9])
tensor([6433, 3867,  368, 2321])
tensor([9534,  666])
tensor([9676,  181,  803,    2, 5886,   11])
tensor([  645, 39429,   661,    44,  1913])
tensor([ 761,    2, 3732,    2,  380,    2, 9482,    2, 2409])
tensor([ 134, 1515, 3229,   91, 1098,    3,    9])
tensor([   55, 11340,     2,  1716,    85,     2,  1291,     2, 42175])
tensor([ 7,  3, 25, 35, 72])
tensor([ 310, 1869,   10, 4039])
tensor([  156, 12251,   252])
tensor([  75, 3487,    3,  152,    3, 1038])
tensor([1928, 22

tensor([ 786,  578,  115, 3901])
tensor([  19, 2539,  106, 4159, 1358,  531,   66,   29,    2,    5, 1518,    6])
tensor([ 1150,  1060,    37, 16190])
tensor([   23,  9950,  6478, 11102,    44])
tensor([  35, 1138,   50,    8,   29,    2])
tensor([   23,  1126,   382,  1316,   265,   804, 13804,     7,     3,   120,
           44])
tensor([   23,   899,    47,     2, 23790])
tensor([ 688,  723, 2090,   53])
tensor([1032])
tensor([1032])
tensor([1032])
tensor([  75,  358, 1780])
tensor([  33, 4291,    8,    3,   15])
tensor([1032])
tensor([1032])
tensor([ 952,   66, 3542,    4, 7329, 7047])
tensor([30390])


,category_id,item_name,Pred
18969645,203,Пакет,114
32644219,84,Пакет,114
37655096,203,Пакет,114
22391292,203,Пакет,114
31856671,203,Пакет,114
...,...,...,...
5113565,203,Пакет,114
35855913,203,Пакет,114
36333025,203,Пакет,114
38930422,203,Пакет,114


::

       This is a Sports news


